# BCG Finance AI Bot - Automated Data Extraction

I wanted to perform data extraction by using two different menthods. 
Usually in companies the data is stored either in databased system or cloud system like AWS, GCP or Postreg.
Here, for this project I wanted to reteive the data from two different souces my first source would be extracting the data from local system in pdf fomrat and scound I'm going to scrap the data from an online website.

My task for data extraction would be: 
1. Extracting the data from interner throught website
2. Downloading some data files from the internet and store into local system in pdf format and then scraping the data from pdf file.
3. Combining this two data files together inorder to get an complete data information files in to an Excel sheet.

In [ ]:
# Imporing All the Necessary Libraries
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [46]:
import requests
import pandas as pd

headers = {"User-Agent": "Madhura Aravendekar madhuraaravendekar29@gmail.com"}

cik = ["0001318605","0000789019","0000320193"]
url = f"https://data.sec.gov/api/xbrl/companyfacts/CIK{cik[0]}.json"

response = requests.get(url, headers=headers)
data = response.json()

facts = data['facts']['us-gaap']
metrics = {
    "Total Revenue": "Revenues",
    "Net Income": "NetIncomeLoss",
    "Total Assets": "Assets",
    "Total Liabilities": "Liabilities",
    "Operating Cash Flow": "NetCashProvidedByUsedInOperatingActivities"
}

targeted_year = [2023,2024,2025]
final_df = pd.DataFrame()

for name, tag in metrics.items():
    if tag in facts:
        df = pd.DataFrame(facts[tag]['units']['USD'])
        df = df[df['form'] == '10-K']
        df['year'] = pd.to_datetime(df['end']).dt.year
        df = df[df['year'].isin(targeted_year)]
        df = df.sort_values('end', ascending=False)
        df = df[['year', 'val']]
        df = df.rename(columns={'val': name})
        df = df.drop_duplicates(subset='year', keep='first')
        if final_df.empty:
            final_df = df
        else:
            final_df = pd.merge(final_df, df, on='year', how='outer')


desired_order = [
    'company',
    'year',
    'Total Revenue',
    'Net Income',
    'Total Assets',
    'Total Liabilities',
    'Operating Cash Flow'
]

final_df = final_df.reindex(columns=desired_order)
final_df['company']= "Tesla"
final_df

,company,year,Total Revenue,Net Income,Total Assets,Total Liabilities,Operating Cash Flow
0,Tesla,2023,96773000000,14997000000,106618000000,43009000000,13256000000
1,Tesla,2024,97690000000,7091000000,122070000000,48390000000,14923000000
2,Tesla,2025,94827000000,3794000000,137806000000,54941000000,14747000000


dict_keys(['AccountsAndNotesReceivableNet', 'AccountsPayableCurrent', 'AccountsReceivableNetCurrent', 'AccretionAmortizationOfDiscountsAndPremiumsInvestments', 'AccrualForEnvironmentalLossContingencies', 'AccrualForEnvironmentalLossContingenciesPayments', 'AccruedEnvironmentalLossContingenciesCurrent', 'AccruedEnvironmentalLossContingenciesNoncurrent', 'AccruedLiabilitiesCurrent', 'AccumulatedDepreciationDepletionAndAmortizationPropertyPlantAndEquipment', 'AccumulatedOtherComprehensiveIncomeLossNetOfTax', 'AdditionalPaidInCapital', 'AdditionalPaidInCapitalCommonStock', 'AdjustmentsNoncashItemsToReconcileNetIncomeLossToCashProvidedByUsedInOperatingActivities', 'AdjustmentsToAdditionalPaidInCapitalEquityComponentOfConvertibleDebt', 'AdjustmentsToAdditionalPaidInCapitalOther', 'AdjustmentsToAdditionalPaidInCapitalSharebasedCompensationRequisiteServicePeriodRecognitionValue', 'AdjustmentsToAdditionalPaidInCapitalTaxEffectFromShareBasedCompensation', 'AdjustmentsToAdditionalPaidInCapitalWar